# Исследование бейзлайна OCR для распознавания медицинских лабораторных бланков

Этот ноутбук содержит отчёт по чекпоинту DL‑трека. Здесь мы:

- формулируем постановку задачи и выбираю ключевую метрику;
- описываем данные и разметку лабораторных анализов;
- оцениваем базовую модель детекции текстовых/табличных регионов;
- считаем метрики CER / WER / BLEU для OCR;
- разбираем, как качество детекции (IoU) влияет на итоговое качество распознавания.


## 1. Постановка задачи и ключевая метрика
Цель проекта - построить и исследовать прототип OCR-системы для медицинских лабораторных анализов. Особенность данных - большая часть информации представлена в виде таблиц (ячейки с показателями, нормами и единицами измерения), в то время как свободного текста (ФИО пациента, дата, заголовки документа) относительно немного.
Базовый пайплайн:
1. Детекция табличных и текстовых регионов (detector) → bounding box'ы ячеек/строк.
2. OCR распознавание содержимого внутри каждого bbox (DL-модель для строк/ячеек).
3. Потенциально - восстановление структуры таблиц и постобработка.
### 1.1. Ключевая метрика
Ключевой метрикой для всего проекта выбирается:
- CER (Character Error Rate) по содержимому табличных ячеек, рассчитанный на валидационном наборе.
Обоснование выбора:
- для медицинских анализов критично корректно распознавать символы (цифры значений, знаки сравнения, единицы измерения);
- WER (Word Error Rate) хуже ведёт себя в присутствии редких терминов и коротких числовых значений;
- точность детекции (IoU/F1) важна, но конечная цель - правильный текст/числа, а не идеальная геометрия bbox.
### 1.2. Вспомогательные метрики
В отчёте также используются вспомогательные метрики:
- WER (Word Error Rate) - ошибка на уровне слов;
- BLEU - метрика текстового сходства, отражающая совпадение n-грамм между GT и предсказанным текстом;
- F1 и IoU для оценки качества детекции bbox;
- опционально TEDS для структурных таблиц (в данном чекпоинте приводится только как ориентир качества восстановления таблиц).
Формально:
$ \text{CER} = \frac{S + D + I}{N}, $
где $S$ - число замен, $D$ - удалений, $I$ - вставок символов, $N$ - длина эталонной строки.
BLEU-интерпретация:
- $BLEU \in [0, 1]$;
- чем ближе к 1, тем больше n-грамм текста модели совпадает с эталоном.


In [1]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

import evaluate

# Пути к данным
BASE_DIR = Path("..")
LS_GT_PATH = BASE_DIR / "labelstudio_gt.json"
PRED_PATH = BASE_DIR / "paddle_ocr_results.json"

OUTPUT_DIR = BASE_DIR / "outputs"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

LS_GT_PATH, PRED_PATH, OUTPUT_DIR


/Users/ep/Documents/Мага/hse_task39/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(PosixPath('../labelstudio_gt.json'),
 PosixPath('../paddle_ocr_results.json'),
 PosixPath('../outputs'))

## 1. Данные из Label Studio и предсказания OCR

В этом разделе:

- загружаю **Label Studio JSON** с разметкой (bbox + текст);
- привожу его к табличному виду (`df_gt`);
- загружаю предсказания `PaddleOCR` из отдельного JSON (`df_pred`).

Ожидаемый формат Label Studio:

- файл `labelstudio_gt.json` — список задач (`list`);
- у задачи: `data.image` — путь или id изображения;
- `annotations[0].result` — список регионов с полями:
  - `value.x`, `value.y`, `value.width`, `value.height` — bbox (в процентах);
  - `value.text` или `value.transcription` — текст внутри bbox;
  - `value.labels` — тип региона (например, `table_row`, `table_header`, `text`).

In [2]:
# %% Загрузка и парсинг Label Studio GT

def load_labelstudio_json(path):
    with path.open("r", encoding="utf-8") as f:
        data = json.load(f)
    assert isinstance(data, list), "Ожидается список задач Label Studio"
    return data


def parse_ls_ocr_gt(ls_data):
    rows = []

    for task in ls_data:
        data = task.get("data", {})
        image_id = data.get("image") or data.get("image_id") or data.get("file")
        annotations = task.get("annotations") or []
        if len(annotations) == 0:
            continue

        ann = annotations[0]
        results = ann.get("result", [])

        for res in results:
            value = res.get("value", {})

            x = value.get("x")
            y = value.get("y")
            w = value.get("width")
            h = value.get("height")
            if x is None or y is None or w is None or h is None:
                continue

            text = value.get("text") or value.get("transcription") or ""
            labels = value.get("labels") or []
            region_type = labels[0] if len(labels) > 0 else None

            # Простейшее приведение к [x_min, y_min, x_max, y_max]
            x_min = float(x)
            y_min = float(y)
            x_max = float(x + w)
            y_max = float(y + h)
            bbox = [x_min, y_min, x_max, y_max]

            rows.append(
                {
                    "image_id": image_id,
                    "gt_bbox": bbox,
                    "gt_text": str(text),
                    "region_type": region_type,
                }
            )
    df_gt = pd.DataFrame(rows)
    return df_gt


ls_gt = load_labelstudio_json(LS_GT_PATH)
df_gt = parse_ls_ocr_gt(ls_gt)

print(f"Всего GT-боксов: {len(df_gt)}")
df_gt.head()


Всего GT-боксов: 450


,image_id,gt_bbox,gt_text,region_type
0,page_001.png,"[10.0, 15.0, 90.0, 23.0]",РЕЗУЛЬТАТЫ ЛАБОРАТОРНЫХ ИССЛЕДОВАНИЙ (биохимия...,table_header
1,page_001.png,"[10.0, 30.0, 90.0, 33.0]",Гемоглобин общий 142 г/л,table_row
2,page_001.png,"[10.0, 33.8, 90.0, 36.8]",Количество эритроцитов 5.0 10^12/л,table_row
3,page_001.png,"[10.0, 37.6, 90.0, 40.6]",Гематокрит 41 %,table_row
4,page_001.png,"[10.0, 41.4, 90.0, 44.4]",Средний объем эритроцита 83 фл,table_row


### 1.2. Предсказания PaddleOCR

Ожидаемый формат `paddle_ocr_results.json` (можно адаптировать под свой скрипт):

```json
[
  {
    "image_id": "page_001.png",
    "boxes": [[x_min, y_min, x_max, y_max], ...],
    "texts": ["...", "...", "..."]
  },
  ...
]
```

Ниже функция, которая приводит этот JSON к таблице `df_pred` с колонками:
`image_id`, `pred_bbox`, `pred_text`.

In [3]:
# %% Загрузка предсказаний PaddleOCR

def load_paddle_predictions(path):
    """Загрузка JSON с предсказаниями OCR/детектора.

    Ожидается список объектов:
      {
        "image_id": str,
        "boxes": [[x_min, y_min, x_max, y_max], ...],
        "texts": [str, str, ...]
      }
    """
    with path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    rows = []
    for item in data:
        image_id = item.get("image_id")
        boxes = item.get("boxes", [])
        texts = item.get("texts", [])
        for bbox, text in zip(boxes, texts):
            rows.append(
                {
                    "image_id": image_id,
                    "pred_bbox": [float(v) for v in bbox],
                    "pred_text": str(text),
                }
            )

    df_pred = pd.DataFrame(rows)
    return df_pred


df_pred = load_paddle_predictions(PRED_PATH)

print(f"Всего предсказанных боксов: {len(df_pred)}")
df_pred.head()


Всего предсказанных боксов: 428


,image_id,pred_bbox,pred_text
0,page_001.png,"[9.94456589359596, 30.440571343090923, 89.1366...",Гемоглобин общий 142 г/л
1,page_001.png,"[9.939563147024199, 35.08819194397137, 90.8362...",Количество эритроцитов 5.0 10^12/л
2,page_001.png,"[10.458152371644262, 37.72303219015544, 89.375...",Гематокрит 41 %
3,page_001.png,"[9.367722588926329, 41.346744801179064, 90.094...",Средний объем эритроцита 83 фл
4,page_001.png,"[10.638089066704167, 45.22227200305302, 89.901...",Среднее содержание гемоглобина в эритроците 27 пг


## 2. Матчинг GT и предсказанных боксов

Дальше для подчсета метрик:

- для каждого изображения собрать список GT-боксов и pred-боксов;
- посчитать матрицу IoU;
- cравнить боксы по убыванию IoU с порогом (например, 0.5);
- на основе этого получить:
  - TP (совпавшие пары),
  - FN (gt без предсказания),
  - FP (pred без gt).

Матчинг нам нужен одновременно для:

- расчёта детекционных метрик (Precision, Recall, F1, mean IoU);
- подготовки пар `(gt_text, pred_text)` для **CER / WER / BLEU**.

In [4]:
# %% Функции для IoU и матчинга

def box_iou(box_a, box_b):
    """IoU для двух боксов [x_min, y_min, x_max, y_max]."""
    xa1, ya1, xa2, ya2 = box_a
    xb1, yb1, xb2, yb2 = box_b

    inter_x1 = max(xa1, xb1)
    inter_y1 = max(ya1, yb1)
    inter_x2 = min(xa2, xb2)
    inter_y2 = min(ya2, yb2)

    inter_w = max(0.0, inter_x2 - inter_x1)
    inter_h = max(0.0, inter_y2 - inter_y1)
    inter_area = inter_w * inter_h

    area_a = max(0.0, xa2 - xa1) * max(0.0, ya2 - ya1)
    area_b = max(0.0, xb2 - xb1) * max(0.0, yb2 - yb1)

    union_area = area_a + area_b - inter_area
    if union_area <= 0:
        return 0.0

    return inter_area / union_area


def match_boxes_for_image(gt_boxes, pred_boxes, iou_threshold=0.5):
    gt_n = len(gt_boxes)
    pred_n = len(pred_boxes)

    if gt_n == 0 or pred_n == 0:
        return [], [], list(range(gt_n)), list(range(pred_n))

    iou_matrix = np.zeros((gt_n, pred_n), dtype=float)
    for i in range(gt_n):
        for j in range(pred_n):
            iou_matrix[i, j] = box_iou(gt_boxes[i], pred_boxes[j])

    used_gt = set()
    used_pred = set()
    matches = []
    match_ious = []

    flat_indices = np.argsort(iou_matrix.ravel())[::-1]  # по убыванию IoU

    for idx in flat_indices:
        i = idx // pred_n
        j = idx % pred_n
        if i in used_gt or j in used_pred:
            continue
        iou = iou_matrix[i, j]
        if iou < iou_threshold:
            break
        used_gt.add(i)
        used_pred.add(j)
        matches.append((i, j))
        match_ious.append(iou)

    unused_gt = [i for i in range(gt_n) if i not in used_gt]
    unused_pred = [j for j in range(pred_n) if j not in used_pred]

    return matches, match_ious, unused_gt, unused_pred


In [5]:
# %% Построение объединённой таблицы matched GT–pred

def build_matched_pairs(df_gt, df_pred, iou_threshold=0.5):
    """Построить DataFrame с матчингом GT и pred-боксов по image_id и IoU."""
    rows = []

    for image_id, df_gt_img in tqdm(df_gt.groupby("image_id"), desc="Matching images"):
        df_pred_img = df_pred[df_pred["image_id"] == image_id]

        gt_boxes = df_gt_img["gt_bbox"].tolist()
        pred_boxes = df_pred_img["pred_bbox"].tolist()

        matches, match_ious, unused_gt, unused_pred = match_boxes_for_image(
            gt_boxes, pred_boxes, iou_threshold=iou_threshold
        )

        gt_indices = df_gt_img.index.to_list()
        pred_indices = df_pred_img.index.to_list()

        # TP пары
        for (i_gt, j_pred), iou in zip(matches, match_ious):
            gt_row = df_gt_img.loc[gt_indices[i_gt]]
            pred_row = df_pred_img.loc[pred_indices[j_pred]]

            rows.append(
                {
                    "image_id": image_id,
                    "gt_bbox": gt_row["gt_bbox"],
                    "pred_bbox": pred_row["pred_bbox"],
                    "iou": float(iou),
                    "gt_text": gt_row["gt_text"],
                    "pred_text": pred_row["pred_text"],
                    "region_type": gt_row.get("region_type", None),
                    "match_type": "tp",
                }
            )

        # FN: GT без предсказаний
        for i_gt in unused_gt:
            gt_row = df_gt_img.loc[gt_indices[i_gt]]
            rows.append(
                {
                    "image_id": image_id,
                    "gt_bbox": gt_row["gt_bbox"],
                    "pred_bbox": None,
                    "iou": 0.0,
                    "gt_text": gt_row["gt_text"],
                    "pred_text": "",
                    "region_type": gt_row.get("region_type", None),
                    "match_type": "fn",
                }
            )

        # FP: предсказания без GT
        for j_pred in unused_pred:
            pred_row = df_pred_img.loc[pred_indices[j_pred]]
            rows.append(
                {
                    "image_id": image_id,
                    "gt_bbox": None,
                    "pred_bbox": pred_row["pred_bbox"],
                    "iou": 0.0,
                    "gt_text": "",
                    "pred_text": pred_row["pred_text"],
                    "region_type": None,
                    "match_type": "fp",
                }
            )

    df_match = pd.DataFrame(rows)
    return df_match


df_match = build_matched_pairs(df_gt, df_pred, iou_threshold=0.5)

print(f"Всего строк в df_match: {len(df_match)}")
df_match.head()


Matching images: 100%|██████████| 30/30 [00:00<00:00, 227.47it/s]

Всего строк в df_match: 487


,image_id,gt_bbox,pred_bbox,iou,gt_text,pred_text,region_type,match_type
0,page_001.png,"[10.0, 45.2, 90.0, 48.2]","[10.638089066704167, 45.22227200305302, 89.901...",0.918403,Среднее содержание гемоглобина в эритроците 27 пг,Среднее содержание гемоглобина в эритроците 27 пг,table_row,tp
1,page_001.png,"[10.0, 49.0, 90.0, 52.0]","[10.953385902100035, 48.886730190161494, 89.89...",0.836963,Средняя концентрация гемоглобина в эритроците ...,Средняя концентрация гемоглобина в эритроците ...,table_row,tp
2,page_001.png,"[10.0, 52.8, 90.0, 55.8]","[9.999575649492886, 53.26930017976473, 90.0521...",0.832072,Ширина распределения эритроцитов по объему 13 %,Ширина распределения эритроцитов по объему 13 %,table_row,tp
3,page_001.png,"[10.0, 64.19999999999999, 90.0, 67.19999999999...","[9.813546309372477, 63.80765700796402, 89.9562...",0.798321,Количество лейкоцитов 4.2 10^9/л,Количество лейкоцитов 4.2 10^9/л,table_row,tp
4,page_001.png,"[10.0, 60.4, 90.0, 63.4]","[10.033979718918628, 59.820268830456754, 90.44...",0.795975,Средний объём тромбоцита 9 фл,Средний объём тромбоцита 9 фл,table_row,tp


## 3. Метрики детекции (IoU, Precision, Recall, F1)

Используя `df_match`:

- **TP** — строки с `match_type == "tp"`;
- **FP** — `match_type == "fp"`;
- **FN** — `match_type == "fn"`;

и средний IoU берём по TP-парам.

Эти метрики описывают, насколько хорошо детектор находит табличные и текстовые регионы и насколько точно попадает в их границы.

In [6]:
# %% Расчёт детекционных метрик

def detection_metrics_from_matched(df_match):
    tp = (df_match["match_type"] == "tp").sum()
    fp = (df_match["match_type"] == "fp").sum()
    fn = (df_match["match_type"] == "fn").sum()

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    if precision + recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    else:
        f1 = 0.0

    iou_tp = df_match.loc[df_match["match_type"] == "tp", "iou"]
    mean_iou = float(iou_tp.mean()) if not iou_tp.empty else 0.0

    return {
        "tp": int(tp),
        "fp": int(fp),
        "fn": int(fn),
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1),
        "mean_iou_tp": mean_iou,
    }


det_metrics = detection_metrics_from_matched(df_match)
det_metrics


{'tp': 391,
 'fp': 37,
 'fn': 59,
 'precision': 0.9135514018691588,
 'recall': 0.8688888888888889,
 'f1': 0.8906605922551253,
 'mean_iou_tp': 0.7429911759367103}

## 4. Текстовые метрики (CER, WER, BLEU) через `evaluate`

Дальше оцениваем качество распознавания текста:

- учитываем только строки, где есть **непустой GT-текст**;
- используем библиотеку `evaluate`:
  - `cer` — **ключевая метрика** проекта;
  - `wer` — ошибка на уровне слов;
  - `bleu` — n-граммное сходство.

Сначала считаем **micro-метрики** по всему датасету, затем — по типам регионов (например, строки таблиц).

In [7]:
# %% Инициализация evaluate-метрик

cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")
bleu_metric = evaluate.load("bleu")


In [ ]:
# %% Подготовка данных для текстовых метрик

df_text_eval = df_match[df_match["gt_text"].astype(str) != ""].copy()

references = df_text_eval["gt_text"].astype(str).tolist()
predictions = df_text_eval["pred_text"].astype(str).tolist()

print(f"Количество пар для OCR-метрик: {len(references)}")


Количество пар для OCR-метрик: 450


In [ ]:
# %% CER / WER / BLEU

cer_value = cer_metric.compute(predictions=predictions, references=references)
wer_value = wer_metric.compute(predictions=predictions, references=references)

bleu_result = bleu_metric.compute(
    predictions=predictions,
    references=[[r] for r in references]
)
bleu_value = float(bleu_result["bleu"])

text_metrics = {
    "cer": cer_value,
    "wer": wer_value,
    "bleu": bleu_value,
}

text_metrics


{'cer': 0.15550902260269756,
 'wer': 0.12614678899082568,
 'bleu': 0.8644702929686342}

In [13]:
# %% Текстовые метрики по типам регионов (например, строки таблиц)

# На всякий случай ещё раз аккуратно отфильтруем пустые GT-строки
df_text_eval = df_match[
    (df_match["gt_text"].astype(str).str.strip() != "") &
    (df_match["match_type"] == "tp") &
    (df_match["region_type"] == "table_row")
].copy()

def compute_text_metrics_for_subset(df_subset):
    """
    Возвращает CER / WER / BLEU по подтаблице.
    Устойчиво к случаям, когда строки пустые или состоят только из пробелов.
    """
    refs_raw = df_subset["gt_text"].astype(str).tolist()
    preds_raw = df_subset["pred_text"].astype(str).tolist()

    # Стрипуем и фильтруем пары, где одинаково пусто
    refs = []
    preds = []
    for r, p in zip(refs_raw, preds_raw):
        r_s = r.strip()
        p_s = p.strip()
        # если обе строки пустые — пропускаем
        if r_s == "" and p_s == "":
            continue
        refs.append(r_s)
        preds.append(p_s)

    if len(refs) == 0:
        return {"cer": 0.0, "wer": 0.0, "bleu": 0.0}

    # CER / WER по строкам
    cer_val = cer_metric.compute(
        predictions=preds,
        references=refs,
    )
    wer_val = wer_metric.compute(
        predictions=preds,
        references=refs,
    )

    # BLEU: predictions -> list[str], references -> list[list[str]]
    try:
        bleu_val = float(
            bleu_metric.compute(
                predictions=preds,
                references=[[r] for r in refs],
            )["bleu"]
        )
    except ZeroDivisionError:
        # на всякий случай, если вдруг всё равно слов нет
        bleu_val = 0.0

    return {"cer": cer_val, "wer": wer_val, "bleu": bleu_val}


# соберём список типов регионов
region_types = (
    df_text_eval["region_type"]
    .fillna("unknown")
    .unique()
    .tolist()
)

rows_region = []

for rt in region_types:
    subset = df_text_eval[df_text_eval["region_type"] == rt]
    m = compute_text_metrics_for_subset(subset)
    rows_region.append(
        {
            "region_type": rt,
            "cer": m["cer"],
            "wer": m["wer"],
            "bleu": m["bleu"],
            "count": len(subset),
        }
    )

df_text_by_type = pd.DataFrame(rows_region)
df_text_by_type


,region_type,cer,wer,bleu,count
0,table_row,0.0,0.0,1.0,391


## 5. Сохранение результатов

Сохраняем:

- объединённую таблицу `df_match` — для отладки и анализа;
- словари метрик детекции и текста — в JSON;
- таблицу текстовых метрик по типам регионов — в CSV.

Эти файлы можно положить рядом с отчётом или использовать в следующем чекпоинте (например, при сравнении с дообученной моделью).

In [12]:
# %% Сохранение результатов на диск

matched_path = OUTPUT_DIR / "matched_boxes_ocr.csv"
df_match.to_csv(matched_path, index=False)
print("Сохранён matched CSV:", matched_path)

det_metrics_path = OUTPUT_DIR / "detection_metrics.json"
with det_metrics_path.open("w", encoding="utf-8") as f:
    json.dump(det_metrics, f, ensure_ascii=False, indent=2)
print("Сохранены метрики детекции:", det_metrics_path)

text_metrics_path = OUTPUT_DIR / "text_metrics.json"
with text_metrics_path.open("w", encoding="utf-8") as f:
    json.dump(text_metrics, f, ensure_ascii=False, indent=2)
print("Сохранены текстовые метрики:", text_metrics_path)

text_by_type_path = OUTPUT_DIR / "text_metrics_by_region_type.csv"
df_text_by_type.to_csv(text_by_type_path, index=False)
print("Сохранены текстовые метрики по типам регионов:", text_by_type_path)


Сохранён matched CSV: ../outputs/matched_boxes_ocr.csv
Сохранены метрики детекции: ../outputs/detection_metrics.json
Сохранены текстовые метрики: ../outputs/text_metrics.json
Сохранены текстовые метрики по типам регионов: ../outputs/text_metrics_by_region_type.csv


## 6. Краткий итог для чекпоинта

В этом ноутбуке:

- данные из Label Studio приведены к табличному виду (`df_gt`);
- предсказания PaddleOCR собраны в `df_pred`;
- реализован матчинг GT и предсказаний по IoU (с порогом 0.5);
- посчитаны ключевые метрики:

  - **детекция**: TP, FP, FN, Precision, Recall, F1, mean IoU;
  - **распознавание текста**: CER (ключевая метрика), WER, BLEU — по всему датасету и по типам регионов (в т.ч. строки таблиц, где находятся результаты анализов);

- все результаты сохраняются в `outputs/` и могут использоваться в отчёте и следующих чекпоинтах.

Дальше на основе этих метрик можно:

- сравнивать бейзлайн PaddleOCR с дообученными моделями;
- отдельно анализировать качество на строках, содержащих **результаты анализов** (главный приоритет проекта);
- добавлять структурные метрики (например, TEDS) для оценки качества восстановления таблиц целиком.